In [ ]:
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='Pricing')#name of the queue in which the server will listen for queries

def priceof(x):
    if x == ord("A"):
        return 10
    elif x == ord("B"):
        return 15
    elif x == ord("C"):
        return 30
    else:
        return "Requested Part is unavailable"

def on_request(ch, method, props, body):
    n = int(body)

    print(" [.] priceof(%x)" % n)
    response = priceof(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag = method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(on_request, queue='Pricing')

print("  Awaiting RPC requests")
channel.start_consuming()
'''firstly the connection channel is opened through using pika package, the channel is declare for connection and 
a queue is dedicated for servers which takes queries from clients and servers listen to the queries through this queue.
After a function on request is called which telles what server has to do after listening the queries, now this funtion calls 
the user defined functions which in turn provides us the response'''


  Awaiting RPC requests
 [.] priceof(41)
 [.] priceof(66)
 [.] priceof(44)
